In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
import hyperopt
import os
import seaborn as sn
from hpsklearn import HyperoptEstimator,random_forest,HyperoptEstimator,any_classifier,any_preprocessing
import hyperopt as hp
from hpsklearn import svc
from sklearn.model_selection import train_test_split
from hyperopt import tpe
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.svm import SVC


## Data prep

In [ ]:
path = "/media/kashraf/Elements/Dissertation/data/preprocessed/visual/psd_data_noisy/"
data1 = pd.read_csv(os.path.join(path,"beta_psd.csv"),index_col=0).iloc[:,:64]
data2 = pd.read_csv(os.path.join(path,"theta_psd.csv"),index_col=0).iloc[:,:64]
data3 = pd.read_csv(os.path.join(path,"alpha_psd.csv"),index_col=0)

data= pd.concat([data1,data2,data3],axis=1)

x=data[data.columns[:64]]
y=data.iloc[:,-1]

scaler=StandardScaler()
x=scaler.fit_transform(x)

pca1=PCA()

pca_data=pca1.fit_transform(x)
components=pca1.components_
explained_var=pca1.explained_variance_ratio_
plt.style.use("seaborn")
plt.figure(figsize=(15,5))
plt.plot(range(len(components)),np.cumsum(explained_var))



In [ ]:
def eigen_sample_select(data,n_comps):
    pca=PCA(n_components=n_comps)
    
    # Fit and project
    projected=pca.fit_transform(data)
    eigen_space=pca.components_
    
    # Renconstuction
    recons=np.dot(projected,eigen_space)
    recon_error=data-recons
    recon_error=np.linalg.norm(recon_error,axis=1)
    
    return recon_error
err=eigen_sample_select(x,40)
    
    

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(15,7))
plt.hist(np.log(err))
plt.xlabel("Log reconstruction error"),plt.ylabel("Number of samples")
# plt.show()
# plt.savefig("Errorhist.jpg")


In [ ]:
def ci_data_selection(errors):
    error=np.log(errors)
    sorted_data=np.sort(error)
    l_limit= np.percentile(sorted_data,25)
    u_limit=np.percentile(sorted_data,75)
    id_selected=[]
    for index , err in enumerate(error):
        if l_limit<=err and err<=u_limit:
            id_selected.append(index)
    return id_selected
selected_index=ci_data_selection(err)

In [ ]:
new_data=pd.DataFrame(data.values[selected_index])
x=new_data.iloc[:,:64]
y=new_data.iloc[:,64]

scaler=StandardScaler()
x=scaler.fit_transform(x)


In [ ]:
x=PCA(n_components=75).fit_transform(x)
x_train,x_test,y_train, y_test= train_test_split(x,y,test_size=0.3)
print("X_train:",x_train.shape)
print("x_test:",x_test.shape)
print("y_train:",y_train.shape)
print("y_test:",y_test.shape)

In [ ]:
estim=HyperoptEstimator(classifier=any_classifier("clf"),algo=tpe.suggest,max_evals=100,
                          trial_timeout=100,n_jobs=12)
estim.fit(x_train,y_train)

print(estim.best_model())

In [ ]:
# model= RandomForestClassifier(bootstrap=True, max_depth=4, max_features='sqrt',
#                        min_samples_leaf=9, n_estimators=10, n_jobs=12,
#                        random_state=0, verbose=False)
model=AdaBoostClassifier(algorithm='SAMME', learning_rate=0.00043987561899226445,
                   n_estimators=12, random_state=3)

# model=SVC(C=10000, cache_size=512, degree=1, gamma='auto',
#     kernel='rbf', max_iter=16883756.0, random_state=2,
#     tol=0.0010625608463669858)
x_train=MinMaxScaler().fit_transform(x_train)
model.fit(x_train,y_train)
x_test=MinMaxScaler().fit_transform(x_test)
y_pred=model.predict(x_test)
print("Report: ", classification_report(y_test,y_pred))

In [ ]:
plt.figure(figsize=(20,10))
conf_mat=confusion_matrix(y_test,y_pred,normalize="true")
labels=["CL-1","CL-2","CL-3","CL-4"]
heat=sn.heatmap(conf_mat,
                annot=True,
                fmt=".2f",
                cmap="Blues",
                xticklabels=labels,
               yticklabels=labels)
# plt.title(titles[5])
# plt.savefig(path+titles[5]+".png")


In [ ]:
confusion_matrix(y_test,y_pred)